In [ ]:
# 1- Importar el archivo csv y extraer las variables Total_Score, Study_hours_per_Week y Attendance(%)
# 2- Limpiar datos aplicando al menos un algoritmo de deteccion de outliers
# 3- Normalizar/estandarizar datos
# 4- Agrupamiento K-means
# 5- Etiquetado y descripcion de clusters 